In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import Intersection_Inspection as ii
import Intersection_Inspection as ii

%matplotlib inline

In [9]:
df = pd.read_csv('data/Crashes_in_DC_larger.csv', low_memory=False)
df.head()

,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,...,BICYCLISTSIMPAIRED,DRIVERSIMPAIRED,TOTAL_TAXIS,TOTAL_GOVERNMENT,SPEEDING_INVOLVED,NEARESTINTROUTEID,NEARESTINTSTREETNAME,OFFINTERSECTION,INTAPPROACHDIRECTION,LOCERROR
0,NaN,NaN,38010550,26960077,16088739,2016-06-02T04:28:03.000Z,NaN,NaN,NaN,NaN,...,0,0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,38010551,26970671,16095587,2016-06-12T15:57:22.000Z,NaN,NaN,NaN,NaN,...,0,0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,38010552,27144599,16204725,2016-12-03T14:06:24.000Z,NaN,NaN,NaN,NaN,...,0,0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,38010553,26719689,15159240,2015-10-08T01:35:59.000Z,NaN,NaN,NaN,NaN,...,0,0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,38010554,26722499,15160560,2015-10-10T09:21:46.000Z,NaN,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


### Clean up the data

In [10]:
df.columns = [x.lower() for x in df.columns]
df.drop(columns=['locerror','todate'], inplace=True)
df.columns

# Fill in some missing x and y values.
df.x = df.longitude
df.y = df.latitude

df.dropna(how='any', axis=0, inplace=True)

# Reset the index in the reduced df so we avoid index out of bounds errors.
df.reset_index(inplace=True, drop=True) # avoid index out of bounds

# Correct an x-variable with the wrong sign (only one instance).
df.loc[df.x > 0]['x'] = df.x * -1

# Convert ward from string to a numeric value
df['ward_number'] = df.ward.str.split(' ', expand=True)[1]
df.ward_number = pd.to_numeric(df.ward_number)

df.isnull().sum()
df.loc[df.ward_number.isnull()][['ward','ward_number']]
df.drop(df.loc[df.ward_number.isnull()].index, axis=0, inplace=True)

In [114]:
foo.columns

Index(['x', 'y', 'objectid', 'crimeid', 'ccn', 'reportdate', 'routeid',
       'measure', 'offset', 'streetsegid', 'roadwaysegid', 'fromdate', 'marid',
       'address', 'latitude', 'longitude', 'xcoord', 'ycoord', 'ward',
       'eventid', 'mar_address', 'mar_score', 'majorinjuries_bicyclist',
       'minorinjuries_bicyclist', 'unknowninjuries_bicyclist',
       'fatal_bicyclist', 'majorinjuries_driver', 'minorinjuries_driver',
       'unknowninjuries_driver', 'fatal_driver', 'majorinjuries_pedestrian',
       'minorinjuries_pedestrian', 'unknowninjuries_pedestrian',
       'fatal_pedestrian', 'total_vehicles', 'total_bicycles',
       'total_pedestrians', 'pedestriansimpaired', 'bicyclistsimpaired',
       'driversimpaired', 'total_taxis', 'total_government',
       'speeding_involved', 'nearestintrouteid', 'nearestintstreetname',
       'offintersection', 'intapproachdirection', 'ward_number', 'closest_pts',
       'this_road_pts', 'other_road_pts', 'whatever'],
      dtype='object'

In [115]:
import math

def get_nearby_pts(row):
    xrows = df.iloc[((df['x']-abs(row.x)) + (df['y'] - abs(row.y))).abs().argsort()[:50]].index.tolist()
    return xrows

# get distance btwn rows
#r1 = df.loc[0]
#r2 = df.loc[1]
#dist = ((r1 - r2))

def get_nearby_pts_foo(row):
    xrows = foo.iloc[((foo['x']-abs(row.x)) + (foo['y'] - abs(row.y))).abs().argsort()[:50]].index.tolist()
    return xrows

foo = df[120000:].copy(deep=True)
foo.shape
#foo.closest_pts = foo.iloc[math.sqrt((foo.x - row.x)**2 + (foo.y - row.y)**2).abs().argsort()[:50]].index.tolist()
#foo.head()

foo.columns
foo['closest_pts'] = foo.apply(get_nearby_pts_foo)
#foo.iloc[((foo.x - row.x)**2 + (foo.y - row.y)**2).abs().argsort()[:5]]
#foo.head()


AttributeError: ("'Series' object has no attribute 'x'", 'occurred at index x')

In [ ]:
'''step = 5000
for i in range(0,125000, step):
    print(i, i+step)
    df['closest_pts'] = df[i:i+step].apply(get_nearby_pts, axis=1)
    
lower = (step*int(df.shape[0]/step))
upper = lower + df.shape[0] % step

df['closest_pts'] = df[lower:upper].apply(get_nearby_pts, axis=1)'''

In [72]:
%time
df['closest_pts'] = df[:5].apply(get_nearby_pts, axis=1)
df.head(15)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


,x,y,objectid,crimeid,ccn,reportdate,routeid,measure,offset,streetsegid,...,speeding_involved,nearestintrouteid,nearestintstreetname,offintersection,intapproachdirection,ward_number,closest_pts,this_road_pts,other_road_pts,whatever
0,-77.012155,38.915726,38011120,26647118,10155168,2010-10-25T05:00:00.000Z,11000102,2709.977,0.0,4878.0,...,0.0,11075462,RHODE ISLAND AVE NW,0.076475,Northeast,5.0,"[63815, 64030, 64012, 63824, 63868, 64106, 103...",0,0,NaN
1,-77.012173,38.892087,38011121,26646859,10155084,2010-10-25T05:00:00.000Z,11000102,210.312,0.0,5223.0,...,0.0,11025352,CONSTITUTION AVE NW,0.042201,North,2.0,"[63815, 64030, 63824, 63868, 64012, 64106, 103...",0,0,0.123
2,-77.011613,38.955586,38011122,27298127,17073723,2017-05-04T02:36:20.000Z,11000102,5010.607,1.0,8240.0,...,0.0,11049922,JEFFERSON ST NW,2.911309,South,4.0,"[63815, 64030, 63824, 63868, 64012, 64106, 103...",0,0,0.123
3,-77.011614,38.956395,38011123,26648369,11171779,2011-11-22T05:00:00.000Z,11000102,5100.218,0.0,10395.0,...,0.0,11052062,KENNEDY ST NW,21.374920,South,4.0,"[63815, 64030, 63824, 63868, 64012, 64106, 103...",0,0,0.123
4,-77.012149,38.919178,38011124,26647593,11168676,2011-11-16T05:00:00.000Z,11000102,3093.110,0.0,10744.0,...,0.0,11090152,W ST NW,0.086942,North,5.0,"[63815, 64030, 64012, 63824, 63868, 64106, 103...",0,0,NaN
5,-77.012164,38.908570,38011125,26647609,11167512,2011-11-14T05:00:00.000Z,11000102,1915.668,0.0,7841.0,...,0.0,11065842,O ST NW,0.070720,North,5.0,NaN,0,0,NaN
6,-77.012378,38.915928,38011126,27237505,17036687,2017-03-05T08:52:26.000Z,11000102,2732.532,0.0,11413.0,...,0.0,11075462,RHODE ISLAND AVE NW,29.633977,Northwest,5.0,NaN,0,0,NaN
7,-77.012160,38.906269,38011127,27247747,17042412,2017-03-15T22:17:53.000Z,11000102,1660.246,0.0,5705.0,...,0.0,11064672,NEW YORK AVE NW,0.041557,North,6.0,NaN,0,0,NaN
8,-77.012089,38.900274,38011128,27250057,17043860,2017-03-18T04:41:59.000Z,11000102,993.953,0.0,15414.0,...,0.0,11042442,H ST NW,11.153248,North,6.0,NaN,0,0,NaN
9,-77.012160,38.906269,38011129,27162706,16215088,2016-12-21T18:32:18.000Z,11000102,1660.246,0.0,5705.0,...,0.0,11064672,NEW YORK AVE NW,0.041557,North,6.0,NaN,0,0,NaN


In [73]:
%%time
def get_nearby_this_and_other(row):
    ''' sort the rows into "this" street and "other" streets by streetsegid.'''
    #nearby_rows = df.iloc[((df['x']-abs(row.x)) + (df['y'] - abs(row.y))).abs().argsort()[:50]]
    print(type(row.closest_pts))
    nearby_rows = df.iloc[row.closest_pts] #row.iloc[row.closest_]
    #print(nearby_rows)
    #df['this_road_pts'] = nearby_rows.loc[nearby_rows.streetsegid == row.streetsegid].index.tolist()
    somelist = nearby_rows.loc[nearby_rows.streetsegid == row.streetsegid].index.tolist()
    print(type(somelist), somelist)
    #df['other_road_pts'] = nearby_rows.loc[nearby_rows.streetsegid != row.streetsegid].index.tolist()
    #return this_rows_list, other_rows_list
    return 0.123

df['this_road_pts'] = 0
df['other_road_pts'] = 0

df['whatever'] = df[1:4].apply(get_nearby_this_and_other, axis=1)

<class 'list'>
<class 'list'> []
<class 'list'>
<class 'list'> []
<class 'list'>
<class 'list'> []
CPU times: user 9 ms, sys: 1.83 ms, total: 10.8 ms
Wall time: 8.08 ms


In [86]:
row = df.iloc[0]
row
nearby_rows =  df.iloc[row.closest_pts]
nearby_rows

# get list of indices that have same streetsegid as 'row;
somelist = nearby_rows.loc[nearby_rows.streetsegid == 7741].index.tolist()
somelist
#otherlist = nearby_rows.loc[nearby_rows.streetsegid != row.streetsegid].index.tolist()
#otherlist

#print (nearby_rows.streetsegid)

[75395, 75345, 75340, 75405, 75334, 75409, 75412, 75397, 75402]

In [16]:
df.columns

Index(['x', 'y', 'objectid', 'crimeid', 'ccn', 'reportdate', 'routeid',
       'measure', 'offset', 'streetsegid', 'roadwaysegid', 'fromdate', 'marid',
       'address', 'latitude', 'longitude', 'xcoord', 'ycoord', 'ward',
       'eventid', 'mar_address', 'mar_score', 'majorinjuries_bicyclist',
       'minorinjuries_bicyclist', 'unknowninjuries_bicyclist',
       'fatal_bicyclist', 'majorinjuries_driver', 'minorinjuries_driver',
       'unknowninjuries_driver', 'fatal_driver', 'majorinjuries_pedestrian',
       'minorinjuries_pedestrian', 'unknowninjuries_pedestrian',
       'fatal_pedestrian', 'total_vehicles', 'total_bicycles',
       'total_pedestrians', 'pedestriansimpaired', 'bicyclistsimpaired',
       'driversimpaired', 'total_taxis', 'total_government',
       'speeding_involved', 'nearestintrouteid', 'nearestintstreetname',
       'offintersection', 'intapproachdirection', 'ward_number',
       'closest_pts'],
      dtype='object')